In [ ]:
# Load data point-to-point
import pandas as pd
import numpy as np
from datetime import datetime

data1 = pd.read_csv("2016-usopen-points.csv")
data2 = pd.read_csv("2016-wimbledon-points.csv")
data3 = pd.read_csv("2017-usopen-points.csv")
data4 = pd.read_csv("2017-wimbledon-points.csv")
data5 = pd.read_csv("2018-usopen-points.csv")
data6 = pd.read_csv("2018-wimbledon-points.csv")
data7 = pd.read_csv("2019-usopen-points.csv")
data8 = pd.read_csv("2019-wimbledon-points.csv")
data9 = pd.read_csv("2020-usopen-points.csv")

dataSamlet = pd.concat([data1, data2, data3, data4, data5, data6, data7, data8, data9])

pd.set_option("display.max_columns", None, "display.max_rows", 300)
dataSamlet = dataSamlet.dropna(axis=1, how='all')

In [ ]:
hala = dataSamlet.loc[dataSamlet['P1DistanceRun'] != 0] #Finds all match_id's with values of ServeWidth, ServeDepth and ReturnDepth

df = pd.DataFrame(columns=hala.columns) #Making empty dataframe

for i in range(len(hala.match_id.unique())): #Adding matches which have values in ServeWidth, ServeDepth and ReturnDepth
    
    df = df.append(dataSamlet.loc[dataSamlet['match_id'] == hala.match_id.unique()[i]])

df = df.reset_index(drop=True) #Reset index

for i in range(len(df)): #Making return balls which the opponent smash on have a ReturnDepth and not NaN
    
    if pd.isna(df.iloc[i]['ReturnDepth']) and df.iloc[i]['RallyCount'] > 1:
        df['ReturnDepth'][i] = 'Service box'
        
for i in range(len(df)): #Giving rows with double fault a value for ServeWidth, ServeDepth and ReturnDepth
    
    if df.iloc[i]['P1DoubleFault'] == 1 or df.iloc[i]['P2DoubleFault'] == 1:
        df['ServeWidth'][i] = 'DoubleFault'
        df['ServeDepth'][i] = 'DoubleFault'
        df['ReturnDepth'][i] = 'DoubleFault'
        
for i in range(len(df)): #Giving rows with WinnerType S a P1Ace or P2Ace
    
    if (df.iloc[i]['WinnerType'] == 'S' and df.iloc[i]['PointServer'] == 1):
        df['P1Ace'][i] = 1
        
    elif (df.iloc[i]['WinnerType'] == 'S' and df.iloc[i]['PointServer'] == 2):
        df['P2Ace'][i] = 1
    
for i in range(len(df)): #An Ace and a serve where the opponent touches it but it doesnt hit the field have the same value.
    
    if df.iloc[i]['RallyCount'] <= 1 and pd.isna(df.iloc[i]['ReturnDepth']) and df.iloc[i]['PointServer'] == 1:
        df['P1Ace'][i] = 1
        
    elif df.iloc[i]['RallyCount'] <= 1 and pd.isna(df.iloc[i]['ReturnDepth']) and df.iloc[i]['PointServer'] == 2:
        df['P2Ace'][i] = 1

        
for i in range(len(df)): #Making serve aces point a ReturnDepth
    
    if df.iloc[i]['P1Ace'] == 1 or df.iloc[i]['P2Ace'] == 1:
        df['ReturnDepth'][i] = 'ServeAce'
        
df = df.reset_index(drop=True)

In [ ]:
for i in range(len(df)): #Making start row for each match with P1Score = 0, P2Score = 0 and PointNumber = 0
    
    if df.iloc[i]['PointNumber'] == '0X':
        
        df = df.drop(i)
        df['PointNumber'][i+1] = '0'
        df = df.reset_index(drop=True)

In [ ]:
df1 = df.copy()

Shifting data

In [ ]:
Forskyd = df1[['ElapsedTime','SetNo','SetWinner','GameNo','GameWinner','PointNumber','PointWinner','PointServer','Speed_KMH','P1Momentum','P2Momentum','P1Ace','P2Ace','P1Winner','P2Winner','P1DoubleFault','P2DoubleFault','P1UnfErr','P2UnfErr','P1NetPoint','P2NetPoint','P1BreakPoint','P2BreakPoint','History','Speed_MPH','ServeIndicator','ServeNumber','WinnerType','WinnerShotType','P1DistanceRun','P2DistanceRun','RallyCount','ServeWidth','ServeDepth','ReturnDepth']].to_numpy()

a = Forskyd[0]

Forskyd = np.delete(Forskyd, 0, 0)

Forskyd = np.vstack([Forskyd, a])

df1[['ElapsedTime','SetNo','SetWinner','GameNo','GameWinner','PointNumber','PointWinner','PointServer','Speed_KMH','P1Momentum','P2Momentum','P1Ace','P2Ace','P1Winner','P2Winner','P1DoubleFault','P2DoubleFault','P1UnfErr','P2UnfErr','P1NetPoint','P2NetPoint','P1BreakPoint','P2BreakPoint','History','Speed_MPH','ServeIndicator','ServeNumber','WinnerType','WinnerShotType','P1DistanceRun','P2DistanceRun','RallyCount','ServeWidth','ServeDepth','ReturnDepth']] = Forskyd

In [ ]:
df1 = df1[df1.PointNumber != 0]
df1 = df1[df1.PointNumber != '0']

# Making accumulated features

In [ ]:
df = df1.drop(df[["P1Momentum", "P2Momentum", "Speed_MPH","History"]], axis=1)
df = df[df.ElapsedTime != "0"]

bad_id=[]
id=np.unique(df['match_id'].to_numpy())

for i in range(len(id)):
    if id[i][-4]=="2":
        bad_id.append(id[i])

df=df[df["match_id"].apply(lambda x:x not in bad_id)]

df.P1Score = df.P1Score.replace({"AD": 50})
df.P2Score = df.P2Score.replace({"AD": 50})
cols=['PointNumber','P1Score', 'P2Score']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

df = df[df.match_id != "2018-wimbledon-1306"]

df['datetime'] = pd.to_datetime(df['ElapsedTime'], format='%H:%M:%S')

df = df.reset_index(drop=True)

time_sec=[]
for i in range(len(df)-1):
    time_sec.append((df['datetime'].iloc[i+1]-df['datetime'][i]).total_seconds())
time_sec.append(-1234)

df['Point_lenght_sec']=time_sec

df_A=df.copy()

In [ ]:
P1_ServeWidth=[]
P1_ServeDepth=[]
P1_ReturnDepth=[]
P2_ServeWidth=[]
P2_ServeDepth=[]
P2_ReturnDepth=[]

cols=['P1_ServeWidth','P1_ServeDepth','P1_ReturnDepth','P2_ServeWidth','P2_ServeDepth','P2_ReturnDepth']

for i in range(len(df_A.match_id.unique())):
        b = df[df['match_id'] == df.match_id.unique()[i]]
        Width=0
        Depth=0
        Return=0
        for j in b.index:
            if b.PointServer[j] == 1:
                P1_ServeWidth.append(b.ServeWidth[j])
                P1_ServeDepth.append(b.ServeDepth[j])
                P1_ReturnDepth.append(b.ReturnDepth[j])
                P2_ServeWidth.append(0)
                P2_ServeDepth.append(0)
                P2_ReturnDepth.append(0)

            elif b.PointServer[j] == 2:
                P2_ServeWidth.append(b.ServeWidth[j])
                P2_ServeDepth.append(b.ServeDepth[j])
                P2_ReturnDepth.append(b.ReturnDepth[j])
                P1_ServeWidth.append(0)
                P1_ServeDepth.append(0)
                P1_ReturnDepth.append(0)
            else:
                P2_ServeWidth.append(0)
                P2_ServeDepth.append(0)
                P2_ReturnDepth.append(0)
                P1_ServeWidth.append(0)
                P1_ServeDepth.append(0)
                P1_ReturnDepth.append(0)


df_A['P1_ServeWidth']=P1_ServeWidth
df_A['P1_ServeDepth']=P1_ServeDepth
df_A['P1_ReturnDepth']=P1_ReturnDepth
df_A['P2_ServeWidth']=P2_ServeWidth
df_A['P2_ServeDepth']=P2_ServeDepth
df_A['P2_ReturnDepth']=P2_ReturnDepth



output = [x for x in pd.get_dummies(df_A,columns=cols).columns if not x in df_A.columns]

In [ ]:
df_A=pd.get_dummies(df_A,columns=cols)

P1_ServeWidth_B_A =[]
P1_ServeWidth_BC_A =[]
P1_ServeWidth_BW_A =[]
P1_ServeWidth_C_A =[]
P1_ServeWidth_DoubleFault_A =[]
P1_ServeWidth_W_A =[]
P1_ServeDepth_CTL_A =[]
P1_ServeDepth_DoubleFault_A =[]
P1_ServeDepth_NCTL_A =[]
P1_ReturnDepth_D_A =[]
P1_ReturnDepth_DoubleFault_A =[]
P1_ReturnDepth_ND_A =[]
P1_ReturnDepth_ServeAce_A =[]
P1_ReturnDepth_Service_box_A =[]
P2_ServeWidth_B_A =[]
P2_ServeWidth_BC_A =[]
P2_ServeWidth_BW_A =[]
P2_ServeWidth_C_A =[]
P2_ServeWidth_DoubleFault_A =[]
P2_ServeWidth_W_A =[]
P2_ServeDepth_CTL_A =[]
P2_ServeDepth_DoubleFault_A =[]
P2_ServeDepth_NCTL_A =[]
P2_ReturnDepth_D_A =[]
P2_ReturnDepth_DoubleFault_A =[]
P2_ReturnDepth_ND_A =[]
P2_ReturnDepth_ServeAce_A =[]
P2_ReturnDepth_Service_box_A =[]


for i in range(len(df.match_id.unique())):
    P1_ServeWidth_B_A = np.append(P1_ServeWidth_B_A, np.insert(df_A[df_A['match_id'] == df.match_id.unique()[i]].P1_ServeWidth_B.cumsum().values[:-1],0,0))
    P1_ServeWidth_BC_A = np.append(P1_ServeWidth_BC_A, np.insert(df_A[df_A['match_id'] == df.match_id.unique()[i]].P1_ServeWidth_BC.cumsum().values[:-1],0,0))
    P1_ServeWidth_BW_A = np.append(P1_ServeWidth_BW_A, np.insert(df_A[df_A['match_id'] == df.match_id.unique()[i]].P1_ServeWidth_BW.cumsum().values[:-1],0,0))
    P1_ServeWidth_C_A = np.append(P1_ServeWidth_C_A, np.insert(df_A[df_A['match_id'] == df.match_id.unique()[i]].P1_ServeWidth_C.cumsum().values[:-1],0,0))
    P1_ServeWidth_DoubleFault_A = np.append(P1_ServeWidth_DoubleFault_A, np.insert(df_A[df_A['match_id'] == df.match_id.unique()[i]].P1_ServeWidth_DoubleFault.cumsum().values[:-1],0,0))
    P1_ServeWidth_W_A = np.append(P1_ServeWidth_W_A, np.insert(df_A[df_A['match_id'] == df.match_id.unique()[i]].P1_ServeWidth_W.cumsum().values[:-1],0,0))
    P1_ServeDepth_CTL_A = np.append(P1_ServeDepth_CTL_A, np.insert(df_A[df_A['match_id'] == df.match_id.unique()[i]].P1_ServeDepth_CTL.cumsum().values[:-1],0,0))
    P1_ServeDepth_DoubleFault_A = np.append(P1_ServeDepth_DoubleFault_A, np.insert(df_A[df_A['match_id'] == df.match_id.unique()[i]].P1_ServeDepth_DoubleFault.cumsum().values[:-1],0,0))
    P1_ServeDepth_NCTL_A = np.append(P1_ServeDepth_NCTL_A, np.insert(df_A[df_A['match_id'] == df.match_id.unique()[i]].P1_ServeDepth_NCTL.cumsum().values[:-1],0,0))
    P1_ReturnDepth_D_A = np.append(P1_ReturnDepth_D_A, np.insert(df_A[df_A['match_id'] == df.match_id.unique()[i]].P1_ReturnDepth_D.cumsum().values[:-1],0,0))
  #P1_ReturnDepth_DoubleFault_A = np.append(P1_ReturnDepth_DoubleFault_A, df_A[df_A['match_id'] == df.match_id.unique()[i]].P1_ReturnDepth_DoubleFault.cumsum().values)
    P1_ReturnDepth_ND_A = np.append(P1_ReturnDepth_ND_A, np.insert(df_A[df_A['match_id'] == df.match_id.unique()[i]].P1_ReturnDepth_ND.cumsum().values[:-1],0,0))
    P1_ReturnDepth_ServeAce_A = np.append(P1_ReturnDepth_ServeAce_A, np.insert(df_A[df_A['match_id'] == df.match_id.unique()[i]].P1_ReturnDepth_ServeAce.cumsum().values[:-1],0,0))
  #P1_ReturnDepth_Service_box_A = np.append(P1_ReturnDepth_Service_box_A, df_A[df_A['match_id'] == df.match_id.unique()[i]].P1_ReturnDepth_Service_box.cumsum().values)
    P2_ServeWidth_B_A = np.append(P2_ServeWidth_B_A, np.insert(df_A[df_A['match_id'] == df.match_id.unique()[i]].P2_ServeWidth_B.cumsum().values[:-1],0,0))
    P2_ServeWidth_BC_A = np.append(P2_ServeWidth_BC_A, np.insert(df_A[df_A['match_id'] == df.match_id.unique()[i]].P2_ServeWidth_BC.cumsum().values[:-1],0,0))
    P2_ServeWidth_BW_A = np.append(P2_ServeWidth_BW_A, np.insert(df_A[df_A['match_id'] == df.match_id.unique()[i]].P2_ServeWidth_BW.cumsum().values[:-1],0,0))
    P2_ServeWidth_C_A = np.append(P2_ServeWidth_C_A, np.insert(df_A[df_A['match_id'] == df.match_id.unique()[i]].P2_ServeWidth_C.cumsum().values[:-1],0,0))
    P2_ServeWidth_DoubleFault_A = np.append(P2_ServeWidth_DoubleFault_A, np.insert(df_A[df_A['match_id'] == df.match_id.unique()[i]].P2_ServeWidth_DoubleFault.cumsum().values[:-1],0,0))
    P2_ServeWidth_W_A = np.append(P2_ServeWidth_W_A, np.insert(df_A[df_A['match_id'] == df.match_id.unique()[i]].P2_ServeWidth_W.cumsum().values[:-1],0,0))
    P2_ServeDepth_CTL_A = np.append(P2_ServeDepth_CTL_A, np.insert(df_A[df_A['match_id'] == df.match_id.unique()[i]].P2_ServeDepth_CTL.cumsum().values[:-1],0,0))
    P2_ServeDepth_DoubleFault_A = np.append(P2_ServeDepth_DoubleFault_A, np.insert(df_A[df_A['match_id'] == df.match_id.unique()[i]].P2_ServeDepth_DoubleFault.cumsum().values[:-1],0,0))
    P2_ServeDepth_NCTL_A = np.append(P2_ServeDepth_NCTL_A, np.insert(df_A[df_A['match_id'] == df.match_id.unique()[i]].P2_ServeDepth_NCTL.cumsum().values[:-1],0,0))
    P2_ReturnDepth_D_A = np.append(P2_ReturnDepth_D_A, np.insert(df_A[df_A['match_id'] == df.match_id.unique()[i]].P2_ReturnDepth_D.cumsum().values[:-1],0,0))
  #P2_ReturnDepth_DoubleFault_A = np.append(P2_ReturnDepth_DoubleFault_A, df_A[df_A['match_id'] == df.match_id.unique()[i]].P2_ReturnDepth_DoubleFault.cumsum().values)
    P2_ReturnDepth_ND_A = np.append(P2_ReturnDepth_ND_A, np.insert(df_A[df_A['match_id'] == df.match_id.unique()[i]].P2_ReturnDepth_ND.cumsum().values[:-1],0,0))
    P2_ReturnDepth_ServeAce_A = np.append(P2_ReturnDepth_ServeAce_A, np.insert(df_A[df_A['match_id'] == df.match_id.unique()[i]].P2_ReturnDepth_ServeAce.cumsum().values[:-1],0,0))
  #P2_ReturnDepth_Service_box_A = np.append(P2_ReturnDepth_Service_box_A, df_A[df_A['match_id'] == df.match_id.unique()[i]].P2_ReturnDepth_Service_box.cumsum().values)

df_A['P1_ServeWidth_B_A'] = P1_ServeWidth_B_A
df_A['P1_ServeWidth_BC_A'] = P1_ServeWidth_BC_A
df_A['P1_ServeWidth_BW_A'] = P1_ServeWidth_BW_A
df_A['P1_ServeWidth_C_A'] = P1_ServeWidth_C_A
df_A['P1_ServeWidth_DoubleFault_A'] = P1_ServeWidth_DoubleFault_A
df_A['P1_ServeWidth_W_A'] = P1_ServeWidth_W_A
df_A['P1_ServeDepth_CTL_A'] = P1_ServeDepth_CTL_A
df_A['P1_ServeDepth_DoubleFault_A'] = P1_ServeDepth_DoubleFault_A
df_A['P1_ServeDepth_NCTL_A'] = P1_ServeDepth_NCTL_A
df_A['P1_ReturnDepth_D_A'] = P1_ReturnDepth_D_A
#df_A['P1_ReturnDepth_DoubleFault_A'] = P1_ReturnDepth_DoubleFault_A
df_A['P1_ReturnDepth_ND_A'] = P1_ReturnDepth_ND_A
df_A['P1_ReturnDepth_ServeAce_A'] = P1_ReturnDepth_ServeAce_A
#df_A['P1_ReturnDepth_Service_box_A'] = P1_ReturnDepth_Service_box_A
df_A['P2_ServeWidth_B_A'] = P2_ServeWidth_B_A
df_A['P2_ServeWidth_BC_A'] = P2_ServeWidth_BC_A
df_A['P2_ServeWidth_BW_A'] = P2_ServeWidth_BW_A
df_A['P2_ServeWidth_C_A'] = P2_ServeWidth_C_A
df_A['P2_ServeWidth_DoubleFault_A'] = P2_ServeWidth_DoubleFault_A
df_A['P2_ServeWidth_W_A'] = P2_ServeWidth_W_A
df_A['P2_ServeDepth_CTL_A'] = P2_ServeDepth_CTL_A
df_A['P2_ServeDepth_DoubleFault_A'] = P2_ServeDepth_DoubleFault_A
df_A['P2_ServeDepth_NCTL_A'] = P2_ServeDepth_NCTL_A
df_A['P2_ReturnDepth_D_A'] = P2_ReturnDepth_D_A
#df_A['P2_ReturnDepth_DoubleFault_A'] = P2_ReturnDepth_DoubleFault_A
df_A['P2_ReturnDepth_ND_A'] = P2_ReturnDepth_ND_A
df_A['P2_ReturnDepth_ServeAce_A'] = P2_ReturnDepth_ServeAce_A
#df_A['P2_ReturnDepth_Service_box_A'] = P2_ReturnDepth_Service_box_A

In [ ]:
P1AceA = []
P2AceA = []
P1WinnerA = []
P2WinnerA = []
P1DoubleFaultA = []
P2DoubleFaultA = []
P1UnfErrA = []
P2UnfErrA = []
P1NetPointA = []
P2NetPointA = []
P1NetPointWonA = []
P2NetPointWonA = []
P1BreakPointA = []
P2BreakPointA = []
P1BreakPointWonA = []
P2BreakPointWonA = []
P1BreakPointMissedA = []
P2BreakPointMissedA = []
P1DistanceRunA = []
P2DistanceRunA = []
RallyCountA = []
Avg_Speed_KMH = []
P1SetsWon=[]
P2SetsWon=[]
P1Momentum = []
P2Momentum = []

for i in range(len(df.match_id.unique())):
    #print(f' i is {i}')
    P1AceA = np.append(P1AceA, np.insert(df[df['match_id'] == df.match_id.unique()[i]].P1Ace.cumsum().values[:-1],0,0))
    P2AceA = np.append(P2AceA, np.insert(df[df['match_id'] == df.match_id.unique()[i]].P2Ace.cumsum().values[:-1],0,0))
    P1WinnerA = np.append(P1WinnerA, np.insert(df[df['match_id'] == df.match_id.unique()[i]].P1Winner.cumsum().values[:-1],0,0))
    P2WinnerA = np.append(P2WinnerA, np.insert(df[df['match_id'] == df.match_id.unique()[i]].P2Winner.cumsum().values[:-1],0,0))
    P1DoubleFaultA = np.append(P1DoubleFaultA, np.insert(df[df['match_id'] == df.match_id.unique()[i]].P1DoubleFault.cumsum().values[:-1],0,0))
    P2DoubleFaultA = np.append(P2DoubleFaultA, np.insert(df[df['match_id'] == df.match_id.unique()[i]].P2DoubleFault.cumsum().values[:-1],0,0))
    P1UnfErrA = np.append(P1UnfErrA, np.insert(df[df['match_id'] == df.match_id.unique()[i]].P1UnfErr.cumsum().values[:-1],0,0))
    P2UnfErrA = np.append(P2UnfErrA, np.insert(df[df['match_id'] == df.match_id.unique()[i]].P2UnfErr.cumsum().values[:-1],0,0))
    P1NetPointA = np.append(P1NetPointA, np.insert(df[df['match_id'] == df.match_id.unique()[i]].P1NetPoint.cumsum().values[:-1],0,0))
    P2NetPointA = np.append(P2NetPointA, np.insert(df[df['match_id'] == df.match_id.unique()[i]].P2NetPoint.cumsum().values[:-1],0,0))
    P1NetPointWonA = np.append(P1NetPointWonA, np.insert(df[df['match_id'] == df.match_id.unique()[i]].P1NetPointWon.cumsum().values[:-1],0,0))
    P2NetPointWonA = np.append(P2NetPointWonA, np.insert(df[df['match_id'] == df.match_id.unique()[i]].P2NetPointWon.cumsum().values[:-1],0,0))
    P1BreakPointA = np.append(P1BreakPointA, np.insert(df[df['match_id'] == df.match_id.unique()[i]].P1BreakPoint.cumsum().values[:-1],0,0))
    P2BreakPointA = np.append(P2BreakPointA, np.insert(df[df['match_id'] == df.match_id.unique()[i]].P2BreakPoint.cumsum().values[:-1],0,0))
    P1BreakPointWonA = np.append(P1BreakPointWonA, np.insert(df[df['match_id'] == df.match_id.unique()[i]].P1BreakPointWon.cumsum().values[:-1],0,0))
    P2BreakPointWonA = np.append(P2BreakPointWonA, np.insert(df[df['match_id'] == df.match_id.unique()[i]].P2BreakPointWon.cumsum().values[:-1],0,0))
    P1BreakPointMissedA = np.append(P1BreakPointMissedA, np.insert(df[df['match_id'] == df.match_id.unique()[i]].P1BreakPointMissed.cumsum().values[:-1],0,0))
    P2BreakPointMissedA = np.append(P2BreakPointMissedA, np.insert(df[df['match_id'] == df.match_id.unique()[i]].P2BreakPointMissed.cumsum().values[:-1],0,0))
    P1DistanceRunA = np.append(P1DistanceRunA, np.insert(df[df['match_id'] == df.match_id.unique()[i]].P1DistanceRun.cumsum().values[:-1],0,0))
    P2DistanceRunA = np.append(P2DistanceRunA, np.insert(df[df['match_id'] == df.match_id.unique()[i]].P2DistanceRun.cumsum().values[:-1],0,0))
    RallyCountA = np.append(RallyCountA, np.insert(df[df['match_id'] == df.match_id.unique()[i]].RallyCount.cumsum().values[:-1],0,0))


    b = df[df['match_id'] == df.match_id.unique()[i]]
    P1SetsWonM = []
    P2SetsWonM = []
    P1MomentumM = []
    P2MomentumM = []
    P1 = 0
    P2 = 0
    P1M = 0
    P2M = 0
    for j in b.index:
        #print(f' j is {j}')

        if b.SetWinner[j] == 1:
            P1 = P1 + 1
            
        elif b.SetWinner[j] == 2:
            P2 = P2 + 1
            
        elif b.PointWinner[j] == 1:
            P1M = P1M + 1
            P2M = 0
            
        elif b.PointWinner[j] == 2:
            P2M = P2M + 1
            P1M = 0
        
        P1SetsWonM = np.append(P1SetsWonM, P1)
        P2SetsWonM = np.append(P2SetsWonM, P2)
        P1MomentumM = np.append(P1MomentumM, P1M)
        P2MomentumM = np.append(P2MomentumM, P2M)
    
    P1SetsWon = np.append(P1SetsWon, np.insert(P1SetsWonM[:-1],0,0))
    P2SetsWon = np.append(P2SetsWon, np.insert(P2SetsWonM[:-1],0,0))
    P1Momentum = np.append(P1Momentum, np.insert(P1MomentumM[:-1],0,0))
    P2Momentum = np.append(P2Momentum, np.insert(P2MomentumM[:-1],0,0))
        

        

df_A['P1AceA'] = P1AceA
df_A['P2AceA'] = P2AceA
df_A['P1WinnerA'] = P1WinnerA
df_A['P2WinnerA'] = P2WinnerA
df_A['P1DoubleFaultA'] = P1DoubleFaultA
df_A['P2DoubleFaultA'] = P2DoubleFaultA
df_A['P1UnfErrA'] = P1UnfErrA
df_A['P2UnfErrA'] = P2UnfErrA
df_A['P1NetPointA'] = P1NetPointA
df_A['P2NetPointA'] = P2NetPointA
df_A['P1NetPointWonA'] = P1NetPointWonA
df_A['P2NetPointWonA'] = P2NetPointWonA
df_A['P1BreakPointA'] = P1BreakPointA
df_A['P2BreakPointA'] = P2BreakPointA
df_A['P1BreakPointWonA'] = P1BreakPointWonA
df_A['P2BreakPointWonA'] = P2BreakPointWonA
df_A['P1BreakPointMissedA'] = P1BreakPointMissedA
df_A['P2BreakPointMissedA'] = P2BreakPointMissedA
df_A['P1DistanceRunA'] = P1DistanceRunA
df_A['P2DistanceRunA'] = P2DistanceRunA
df_A['RallyCountA'] = RallyCountA
df_A['P1SetsWon'] = P1SetsWon
df_A['P2SetsWon'] = P2SetsWon
df_A['P1Momentum'] = P1Momentum
df_A['P2Momentum'] = P2Momentum

set_value=[]
game_value=[]
for i in range(l A)-1, 0, -1):
    if df_A.SetWinner.iloc[i]==0:
        set_value.append(set_value[-1])
        
    if df_A.SetWinner.iloc[i]!=0:
        set_value.append(df_A.SetWinner.iloc[i])
        
    if df_A.GameWinner.iloc[i]==0:
        game_value.append(game_value[-1])
                          
    if df_A.GameWinner.iloc[i]!=0:
        game_value.append(df_A.GameWinner.iloc[i])

game_value = game_value[::-1]
game_value.insert(1,1)
#game_value.insert(1,1)

set_value = set_value[::-1]
set_value.insert(1,1)
#set_value.insert(1,1)        

df_A['GameWinnerA'] = game_value
df_A['SetWinnerA'] = set_value

In [ ]:
df_A.loc[df['Point_lenght_sec'] < 0, 'Point_lenght_sec'] = 0

# Adding player rank

In [ ]:
#Load data match_atp
match2 = pd.read_csv("2016-usopen-matches.csv")
match1 = pd.read_csv("2016-wimbledon-matches.csv")
match4 = pd.read_csv("2017-usopen-matches.csv")
match3 = pd.read_csv("2017-wimbledon-matches.csv")
match6 = pd.read_csv("2018-usopen-matches.csv")
match5 = pd.read_csv("2018-wimbledon-matches.csv")
match8 = pd.read_csv("2019-usopen-matches.csv")
match7 = pd.read_csv("2019-wimbledon-matches.csv")
match9 = pd.read_csv("2020-usopen-matches.csv")

matchSamlet = pd.concat([match1,match2,match3,match4,match5,match6,match7,match8,match9])

rank1 = pd.read_csv("atp_matches_2016.csv")
rank2 = pd.read_csv("atp_matches_2017.csv")
rank3 = pd.read_csv("atp_matches_2018.csv")
rank4 = pd.read_csv("atp_matches_2019.csv")
rank5 = pd.read_csv("atp_matches_2020.csv")

rankSamlet = pd.concat([rank1,rank2,rank3,rank4,rank5])

In [ ]:
bad_id=[]
id=np.unique(matchSamlet['match_id'].to_numpy())

for i in range(len(id)):
    if id[i][-4]=="2":
        bad_id.append(id[i])

matchSamlet=matchSamlet[matchSamlet["match_id"].apply(lambda x:x not in bad_id)]

matchSamlet = matchSamlet[matchSamlet.match_id != "2018-wimbledon-1306"]

matchSamlet = matchSamlet.drop(matchSamlet[['status','winner','event_name','round','court_name','court_id','player1id','player2id','nation1','nation2']], axis=1)

matchSamlet.slam = matchSamlet.slam.replace({"wimbledon": 'Wimbledon'})
matchSamlet.slam = matchSamlet.slam.replace({"usopen": 'US Open'})

matchSamlet.player1 = matchSamlet.player1.replace({'Juan Martin del Potro': 'Juan Martin Del Potro'})
matchSamlet.player2 = matchSamlet.player2.replace({'Juan Martin del Potro': 'Juan Martin Del Potro'})
matchSamlet.player1 = matchSamlet.player1.replace({'Alex de Minaur': 'Alex De Minaur'})
matchSamlet.player2 = matchSamlet.player2.replace({'Alex de Minaur': 'Alex De Minaur'})

matchSamlet = matchSamlet.reset_index(drop=True)

In [ ]:
rankSamlet = rankSamlet.drop(rankSamlet[['tourney_id','draw_size','tourney_level', 'winner_entry',
                                          'winner_id','winner_seed','winner_ht','winner_ioc','winner_age','loser_id',
                                          'loser_seed','loser_entry','loser_ht','loser_ioc','loser_age','score','best_of',
                                          'round','minutes','w_ace','w_df','w_svpt','w_1stIn','w_1stWon','w_2ndWon','w_SvGms',
                                          'w_bpSaved','w_bpFaced','l_ace','l_df','l_svpt','l_1stIn','l_1stWon','l_2ndWon',
                                          'l_SvGms','l_bpSaved','l_bpFaced','winner_rank_points','loser_rank_points']], axis=1)

rankSamlet.tourney_name = rankSamlet.tourney_name.replace({"Us Open": 'US Open'})

rankSamlet.winner_name = rankSamlet.winner_name.replace({'Jo-Wilfried Tsonga': 'Jo Wilfried Tsonga'})
rankSamlet.loser_name = rankSamlet.loser_name.replace({'Jo-Wilfried Tsonga': 'Jo Wilfried Tsonga'})
rankSamlet.winner_name = rankSamlet.winner_name.replace({'Juan Martin del Potro': 'Juan Martin Del Potro'})
rankSamlet.loser_name = rankSamlet.loser_name.replace({'Juan Martin del Potro': 'Juan Martin Del Potro'})
rankSamlet.winner_name = rankSamlet.winner_name.replace({'Albert Ramos': 'Albert Ramos Vinolas'})
rankSamlet.loser_name = rankSamlet.loser_name.replace({'Albert Ramos': 'Albert Ramos Vinolas'})
rankSamlet.winner_name = rankSamlet.winner_name.replace({'Alex de Minaur': 'Alex De Minaur'})
rankSamlet.loser_name = rankSamlet.loser_name.replace({'Alex de Minaur': 'Alex De Minaur'})
rankSamlet.winner_name = rankSamlet.winner_name.replace({'Jc Aragone': 'JC Aragone'})
rankSamlet.loser_name = rankSamlet.loser_name.replace({'Jc Aragone': 'JC Aragone'})
rankSamlet.winner_name = rankSamlet.winner_name.replace({'Soon Woo Kwon': 'Soonwoo Kwon'})
rankSamlet.loser_name = rankSamlet.loser_name.replace({'Soon Woo Kwon': 'Soonwoo Kwon'})
rankSamlet.winner_name = rankSamlet.winner_name.replace({'Christian Garin': 'Cristian Garin'})
rankSamlet.loser_name = rankSamlet.loser_name.replace({'Christian Garin': 'Cristian Garin'})

rankSamlet = rankSamlet[(rankSamlet.tourney_name == 'Wimbledon') | (rankSamlet.tourney_name == 'US Open')]

rankSamlet = rankSamlet.reset_index(drop=True)

In [ ]:
for i in range(len(matchSamlet)):
    
    if 100 < matchSamlet.match_num[i] < 1200:
        matchSamlet['match_num'][i] =  matchSamlet['match_num'][i] - 1001
    
    if 1200 < matchSamlet.match_num[i] < 1300:
        matchSamlet['match_num'][i] =  matchSamlet['match_num'][i] - 1037
        
    if 1300 < matchSamlet.match_num[i] < 1400:
        matchSamlet['match_num'][i] =  matchSamlet['match_num'][i] - 1105
        
    if 1400 < matchSamlet.match_num[i] < 1500:
        matchSamlet['match_num'][i] =  matchSamlet['match_num'][i] - 1189
        
    if 1500 < matchSamlet.match_num[i] < 1600:
        matchSamlet['match_num'][i] =  matchSamlet['match_num'][i] - 1281
    
    if 1600 < matchSamlet.match_num[i] < 1700:
        matchSamlet['match_num'][i] =  matchSamlet['match_num'][i] - 1377
    
    if 1700 < matchSamlet.match_num[i] < 1800:
        matchSamlet['match_num'][i] =  matchSamlet['match_num'][i] - 1475

In [ ]:
a = pd.merge(matchSamlet,rankSamlet, left_on='match_num', right_on='match_num',how='left')
a = a.reset_index(drop=True)

In [ ]:
slettet = []
for i in range(len(a)):
    if (a.player1[i] == a.winner_name[i] and a.player2[i] == a.loser_name[i] and a.slam[i] == a.tourney_name[i]) or (a.player2[i] == a.winner_name[i] and a.player1[i] == a.loser_name[i] and a.slam[i] == a.tourney_name[i]):
        continue
    else:
        #print(a.match_id[i])
        #a = a.drop([i])
        slettet.append(i)
        
a = a.drop(slettet)
        
a = a.reset_index(drop=True)

a = a.drop([384,811,927])#Bad data

a = a.reset_index(drop=True)

In [ ]:
df_A_PR = pd.merge(df_A,a,left_on='match_id',right_on='match_id',how='left')
df_A_PR = df_A_PR[df_A_PR.match_id != "2017-usopen-1303"]
df_A_PR = df_A_PR[df_A_PR.match_id != "2020-usopen-1149"]
df_A_PR = df_A_PR[df_A_PR.match_id != "2020-usopen-1155"]
df_A_PR = df_A_PR.reset_index(drop=True)

In [ ]:
total_time=[]
for i in range(len(df_A_PR.match_id.unique())):
    total_time = np.append(total_time, np.insert(df_A_PR[df_A_PR['match_id'] == df_A_PR.match_id.unique()[i]].Point_lenght_sec.cumsum().values[:-1],0,0))
df_A_PR['Total_time']=total_time

Columns to be swapped

In [ ]:
to_be_shifted=[
    ('P1GamesWon','P2GamesWon'),
    ('P1Score','P2Score'),
    ('P1PointsWon','P2PointsWon'),
    ('P1_ServeWidth_B_A','P2_ServeWidth_B_A'),
    ('P1_ServeWidth_BC_A','P2_ServeWidth_BC_A'),
    ('P1_ServeWidth_BW_A','P2_ServeWidth_BW_A'),
    ('P1_ServeWidth_C_A','P2_ServeWidth_C_A'),
    ('P1_ServeWidth_W_A','P2_ServeWidth_W_A'),
    ('P1_ServeDepth_CTL_A','P2_ServeDepth_CTL_A'),
    ('P1_ServeDepth_NCTL_A','P2_ServeDepth_NCTL_A'),
    ('P1_ReturnDepth_D_A','P2_ReturnDepth_D_A'),
    ('P1_ReturnDepth_ND_A','P2_ReturnDepth_ND_A'),
    ('P1AceA','P2AceA'),
    ('P1WinnerA','P2WinnerA'),
    ('P1DoubleFaultA','P2DoubleFaultA'),
    ('P1UnfErrA','P2UnfErrA'),
    ('P1NetPointA','P2NetPointA'),
    ('P1NetPointWonA','P2NetPointWonA'),
    ('P1BreakPointA','P2BreakPointA'),
    ('P1BreakPointWonA','P2BreakPointWonA'),
    ('P1BreakPointMissedA','P2BreakPointMissedA'),
    ('P1DistanceRunA','P1DistanceRunA'),
    ('P1SetsWon','P2SetsWon'),
    ('player1','player2'),
    ('P1NetPoint','P2NetPoint'),
    ('P1Rank','P2Rank')
    ]

Changing pointwinenr to binary encoding 0 = p1, 1= p2

In [ ]:
Server = {1: 0,2: 1}
df_A_PR.PointServer = [Server[item] for item in df_A_PR.PointServer]
df_A_PR.PointWinner = [Server[item] for item in df_A_PR.PointWinner]
df_A_PR.GameWinnerA = [Server[item] for item in df_A_PR.GameWinnerA]
df_A_PR.SetWinnerA = [Server[item] for item in df_A_PR.SetWinnerA]

Surface attribute, 0 = US open (Turf), 1 = Wimbeldon grass

In [ ]:
surface=[]
for i in range(len(df_A_PR)):
    if df_A_PR.match_id.iloc[i][5] == "u":
        surface.append(0)
    else:
        surface.append(1)
df_A_PR['Surface']=surface

Editing point number mistakes in data

In [ ]:
for i in range(len(df_A_PR)):
    if pd.isna(df_A_PR.iloc[i]['PointNumber']):
        df_A_PR['PointNumber'].iloc[i]=df_A_PR['PointNumber'].iloc[i+1]-1

Editing NA value for loser rank in a single match

In [ ]:
df_A_PR.loc[df_A_PR['match_id'] == "2020-usopen-1228", 'loser_rank'] = 501

Making rank for p1 and p2

In [ ]:
p1_rank = []
p2_rank = []
for i in range(len(df_A_PR)):
    if df_A_PR.player1.iloc[i] == df_A_PR.winner_name.iloc[i]:
        p1_rank.append(df_A_PR.winner_rank.iloc[i])
        p2_rank.append(df_A_PR.loser_rank.iloc[i])
    if df_A_PR.player1.iloc[i] == df_A_PR.loser_name.iloc[i]:
        p1_rank.append(df_A_PR.loser_rank.iloc[i])
        p2_rank.append(df_A_PR.winner_rank.iloc[i])
df_A_PR['P1Rank'] = p1_rank
df_A_PR['P2Rank'] = p2_rank

Swapping columns

In [ ]:
df2=df_A_PR.copy()
for i in range(len(df_A_PR)):
    if df_A_PR['PointServer'].loc[i] == 1:
        for j in range(len(to_be_shifted)):
            df_A_PR.at[i,to_be_shifted[j][0]] = df2[to_be_shifted[j][1]].loc[i]
            df_A_PR.at[i,to_be_shifted[j][1]] = df2[to_be_shifted[j][0]].loc[i]
        if df_A_PR.PointWinner.loc[i] == 0:
            df_A_PR.at[i,"PointWinner"] = 1
        elif df_A_PR.PointWinner.loc[i] == 1:
            df_A_PR.at[i,"PointWinner"] = 0
            
        if df_A_PR.GameWinnerA.loc[i] == 0:
            df_A_PR.at[i,"GameWinnerA"]=1
        elif df_A_PR.GameWinnerA.loc[i] == 1: 
            df_A_PR.at[i,"GameWinnerA"] = 0

        if df_A_PR.SetWinnerA.loc[i] == 0:
            df_A_PR.at[i,"SetWinnerA"]=1
        elif df_A_PR.SetWinnerA.loc[i] == 1: 
            df_A_PR.at[i,"SetWinnerA"] = 0
            
    

In [ ]:
tie=[]
for i in range(len(df_A_PR)):
    if (df_A_PR.GameNo.iloc[i]>12) and (df_A_PR.Surface.iloc[i]==0):
        tie.append(1)
    elif (df_A_PR.GameNo.iloc[i] == 25) and (df_A_PR.Surface.iloc[i]==1) and (df_A_PR.year.iloc[i]>2018):
        tie.append(1)
    elif (df_A_PR.GameNo.iloc[i] > 12) and (df_A_PR.Surface.iloc[i]==1) and (df_A_PR.SetNo.iloc[i] < 5):
        tie.append(1)
    else:
        tie.append(0)
df_A_PR['Tiebreak']=tie

In [ ]:
#df_A_PR.to_csv("FinalData.csv")

In [ ]:
# Load data point-to-point
import pandas as pd
import numpy as np
from datetime import datetime
pd.set_option("display.max_columns", None, "display.max_rows", 300)

df=pd.read_csv('FinalData.csv')

In [ ]:
df[['match_id','ElapsedTime','SetNo','GameNo','GameWinner','PointWinner','P1Score','P2Score','P1UnfErr','P2UnfErr', 'P1UnfErrA','P2UnfErrA']][:10]

In [ ]:
Server = {0: 1,1: 2}
df.PointServer = [Server[item] for item in df.PointServer]
df.PointWinner = [Server[item] for item in df.PointWinner]
df.GameWinnerA = [Server[item] for item in df.GameWinnerA]
df.SetWinnerA = [Server[item] for item in df.SetWinnerA]

In [ ]:
Hala = {0: 0,1: 1, 2: 1}
df.GameWinner = [Hala[item] for item in df.GameWinner]
df.SetWinner = [Hala[item] for item in df.SetWinner]

In [ ]:
df[['match_id','ElapsedTime','SetNo','GameNo','GameWinner','GameWinnerA','PointWinner','P1Score','P2Score']][:14]

In [ ]:
df[['match_id','ElapsedTime','SetNo','SetWinner','SetWinnerA','PointWinner','P1Score','P2Score']][60:70]